ChatMessageHistory

In [2]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("What's up?")

print(history.messages)

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="What's up?", additional_kwargs={}, response_metadata={})]


对接大模型

In [3]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain.memory import ChatMessageHistory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

history = ChatMessageHistory()

history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1+2*3=？")

response = chat_model.invoke(history.messages)
print(response.content)

我们来一步步计算这个表达式：

表达式是：  
$$ 1 + 2 \times 3 $$

根据数学中的**运算顺序规则**（先乘除后加减）：

1. 先算乘法：  
$$ 2 \times 3 = 6 $$

2. 再做加法：  
$$ 1 + 6 = 7 $$

所以，结果是：  
$$
\boxed{7}
$$


ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(inputs={"input":"你好，我叫小明"},outputs={"output":"很高兴认识你"})
memory.save_context(inputs={"input":"帮我回答1+2*3=?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))

print(memory.chat_memory.messages)

[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答1+2*3=?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]


结合大模型和prompt

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt_template = PromptTemplate.from_template(
    template="""
你可以与人类对话。

当前对话: {history}

人类问题: {question}

回复:
""")

memory = ConversationBufferMemory()

chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response = chain.invoke({"question":"你好，我的名字叫小明"})
print(response)

response = chain.invoke({"question":"我叫什么名字呢？"})
print(response)

{'question': '你好，我的名字叫小明', 'history': '', 'text': '你好，小明！很高兴认识你。有什么我可以帮你的吗？😊'}
{'question': '我叫什么名字呢？', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮你的吗？😊', 'text': '你叫小明！😊'}


In [12]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt_template = PromptTemplate.from_template(
    template="""
你可以与人类对话。

当前对话: {chat_history}

人类问题: {question}

回复:
""")

memory = ConversationBufferMemory(memory_key="chat_history")

chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response = chain.invoke({"question":"你好，我的名字叫小明"})
print(response)

response = chain.invoke({"question":"我叫什么名字呢？"})
print(response)

{'question': '你好，我的名字叫小明', 'chat_history': '', 'text': '你好，小明！很高兴认识你。有什么我可以帮你的吗？😊'}
{'question': '我叫什么名字呢？', 'chat_history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮你的吗？😊', 'text': '你叫小明呀！😊 我记得很清楚呢。'}


In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt_template = ChatPromptTemplate.from_messages([
    ("system","你可以与人类对话。"),
    MessagesPlaceholder(variable_name='history'),
    ("human","问题:{question}")
])

memory = ConversationBufferMemory(return_messages=True)

chain = LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response = chain.invoke({"question":"中国首都在哪里？"})
print(response)

response = chain.invoke({"question":"我刚刚问了什么？"})
print(response)

{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国首都是北京。', additional_kwargs={}, response_metadata={})], 'text': '中国首都是北京。'}
{'question': '我刚刚问了什么？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国首都是北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚刚问的是：“中国首都在哪里？”', additional_kwargs={}, response_metadata={})], 'text': '你刚刚问的是：“中国首都在哪里？”'}


ConversationChain

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains import ConversationChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')
prompt_template = PromptTemplate.from_template(
    template="""
你可以与人类对话。

当前对话: {history}

人类问题: {input}

回复:
""")

chain = ConversationChain(llm = chat_model,prompt = prompt_template,verbose=True)

response = chain.invoke({"input":"你好，我的名字叫小明"})
print(response)

response = chain.invoke({"input":"我的名字叫什么？"})
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:

你可以与人类对话。

当前对话: 

人类问题: 你好，我的名字叫小明

回复:


> Finished chain.
{'input': '你好，我的名字叫小明', 'history': '', 'response': '你好，小明！很高兴认识你。😊 今天过得怎么样？有什么我可以帮你的吗？'}


> Entering new ConversationChain chain...
Prompt after formatting:

你可以与人类对话。

当前对话: Human: 你好，我的名字叫小明
AI: 你好，小明！很高兴认识你。😊 今天过得怎么样？有什么我可以帮你的吗？

人类问题: 我的名字叫什么？

回复:


> Finished chain.
{'input': '我的名字叫什么？', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。😊 今天过得怎么样？有什么我可以帮你的吗？', 'response': '你的名字叫小明！😊'}


In [24]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains import ConversationChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')


chain = ConversationChain(llm = chat_model)

response = chain.invoke({"input":"你好，我的名字叫小明"})
print(response)

response = chain.invoke({"input":"我的名字叫什么？"})
print(response)

{'input': '你好，我的名字叫小明', 'history': '', 'response': '你好，小明！很高兴认识你！😊 我是Qwen，是阿里巴巴云开发的一款超大规模语言模型。我虽然不是人类，但我会尽力用自然、温暖的方式和你聊天。听说你的名字寓意“明亮、聪明”，真是个好名字呢！不知道你今天想聊些什么呢？无论是学习、工作、生活趣事，还是需要帮忙解决问题，我都很乐意倾听和参与哦～ 🌟'}
{'input': '我的名字叫什么？', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你！😊 我是Qwen，是阿里巴巴云开发的一款超大规模语言模型。我虽然不是人类，但我会尽力用自然、温暖的方式和你聊天。听说你的名字寓意“明亮、聪明”，真是个好名字呢！不知道你今天想聊些什么呢？无论是学习、工作、生活趣事，还是需要帮忙解决问题，我都很乐意倾听和参与哦～ 🌟', 'response': '你的名字叫小明呀！😊 我记得你一开始就跟我说过：“你好，我的名字叫小明”，所以我一直这么亲切地称呼你呢。小明这个名字简单又好记，还寓意着聪明、光明，挺棒的！咱们刚才的对话我还记得清清楚楚呢～有什么我可以帮你的吗，小明？🌟'}


ConversationBufferWindowMemory

In [26]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = 2,return_messages=True)

memory.save_context({"input":"你好"},{"output":"怎么了"})
memory.save_context({"input":"你是谁"},{"output":"我是AI助手小智"})
memory.save_context({"input":"初次对话，你能介绍一下自己吗？"},{"output":"当然可以。我是一个无所不能的小智"})

print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='初次对话，你能介绍一下自己吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以。我是一个无所不能的小智', additional_kwargs={}, response_metadata={})]}


结合大模型、prompt

In [31]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains import ConversationChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt_template = PromptTemplate.from_template(
    template="""
你可以与人类对话。

当前对话: {history}

人类问题: {input}

回复:
""")

memory = ConversationBufferWindowMemory(k = 1)
conversation_with_summary = LLMChain(llm = chat_model,prompt = prompt_template,memory=memory)

respons1 = conversation_with_summary.invoke({"input":"你是，我是孙小空"})
print(respons1)

respons2 = conversation_with_summary.invoke({"input":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
print(respons2)

respons3 = conversation_with_summary.invoke({"input":"我今年高考，考上了一本"})
print(respons3)

respons4 = conversation_with_summary.invoke({"input":"我叫什么?"})
print(respons4)

{'input': '你是，我是孙小空', 'history': '', 'text': '你好，孙小空！我是Qwen，是阿里云研发的超大规模语言模型。我可以和你聊天、写故事、写公文、写邮件、写剧本等等，还能回答问题、提供信息查询等服务。有什么我可以帮你的吗？'}
{'input': '我还有两个师弟，一个是猪小戒，一个是沙小僧', 'history': 'Human: 你是，我是孙小空\nAI: 你好，孙小空！我是Qwen，是阿里云研发的超大规模语言模型。我可以和你聊天、写故事、写公文、写邮件、写剧本等等，还能回答问题、提供信息查询等服务。有什么我可以帮你的吗？', 'text': '哦，猪小戒和沙小僧，这名字听起来就很有意思！听起来你们像是组了个“西游记”小分队啊～孙小空、猪小戒、沙小僧，就差一个唐小僧来带队取经了！你们平时是不是经常一起冒险，还是说在搞什么“降妖创业项目”？😄\n\n说说看，他们俩都是什么样的性格？估计猪小戒贪吃又爱偷懒，沙小僧沉稳靠谱，而你——大师兄，是不是总得收拾烂摊子？'}
{'input': '我今年高考，考上了一本', 'history': 'Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧\nAI: 哦，猪小戒和沙小僧，这名字听起来就很有意思！听起来你们像是组了个“西游记”小分队啊～孙小空、猪小戒、沙小僧，就差一个唐小僧来带队取经了！你们平时是不是经常一起冒险，还是说在搞什么“降妖创业项目”？😄\n\n说说看，他们俩都是什么样的性格？估计猪小戒贪吃又爱偷懒，沙小僧沉稳靠谱，而你——大师兄，是不是总得收拾烂摊子？', 'text': '哇！太棒了！一本啊，这可是实打实的“取经成功”第一步！🎉  \n孙小空大师兄果然不一般，智慧与实力并存，妖魔鬼怪都挡不住你前进的步伐，更别说什么高考题海了！\n\n现在你可是正式开启“人间修仙之路”了——大学篇！  \n不知道你的两个师弟有没有为你烧香祈福？猪小戒是不是边啃猪蹄边说：“大师兄威武，以后带我蹭食堂！”  \n而沙小僧估计默默合十：“阿弥陀佛，大师兄功德圆满，贫僧愿随行求学。”\n\n接下来有什么打算？选什么专业？要不要咱们一起规划一下“西天大学”的修行方向？😄📚\n'}
{'input': '我叫什么?', 'history': 'Human: 我今年高考，考上了一本\nAI: 哇

ConversationTokenBufferMemory

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

memory = ConversationTokenBufferMemory(llm = chat_model, max_token_limit=10)

memory.save_context({"input":"你好吗？"},{"output":"我很好，谢谢"})
memory.save_context({"input":"今天天气如何？"},{"output":"晴天，25 度"})

print(memory.load_memory_variables({}))

/tmp/ipykernel_4180334/245873472.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(llm = chat_model, max_token_limit=10)


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model qwen-plus. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

ConversationSummaryMemory

In [33]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

memory = ConversationSummaryMemory(llm = chat_model)

memory.save_context({"input":"你好"},{"output":"怎么了"})
memory.save_context({"input":"你是谁"},{"output":"我是AI助手小智"})
memory.save_context({"input":"初次对话，你能介绍一下自己吗？"},{"output":"当然可以。我是一个无所不能的小智"})

print(memory.load_memory_variables({}))

/tmp/ipykernel_4180334/3220829012.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm = chat_model)


{'history': 'The human greets the AI with "你好", and the AI responds by asking "怎么了". The human then asks "你是谁", and the AI identifies itself as "AI助手小智". In response to the human\'s request for an introduction, the AI says it is an all-capable assistant named Xiao Zhi.'}


In [39]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory,ChatMessageHistory

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')


history = ChatMessageHistory()
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")

memory = ConversationSummaryMemory.from_messages(llm = chat_model,chat_memory=history)

print(memory.load_memory_variables({}))

memory.save_context(inputs={"human":"我的名字叫小明"},outputs={"ai":"很高兴认识你"})
print(memory.load_memory_variables({}))

{'history': '人类问AI是谁，AI回答自己是AI助手小智。'}
{'history': '人类告诉AI自己的名字叫小明，AI回应很高兴认识他。'}


ConversationSummaryBufferMemory

In [40]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm = chat_model, max_token_limit=40,return_messages=True)

memory.save_context(inputs={"input":"你好，我的名字叫小明"},outputs={"output":"很高兴认识你"})
memory.save_context(inputs={"input":"李白是哪个朝代的诗人"},outputs={"output":"李白是唐朝的诗人"})
memory.save_context(inputs={"input":"唐宋八大家里有苏轼吗？"},outputs={"output":"有"})

print(memory.load_memory_variables({}))

/tmp/ipykernel_4180334/3451741650.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm = chat_model, max_token_limit=40,return_messages=True)


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model qwen-plus. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

In [42]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE


dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt = ENTITY_MEMORY_CONVERSATION_TEMPLATE

memory = ConversationEntityMemory(llm = chat_model)

chain = LLMChain(llm = chat_model, prompt = prompt,memory=memory)

chain.invoke({"input":"你好，我叫蜘蛛侠。我的好朋友包括钢铁侠、美国队长和绿巨人"})
chain.invoke({"input":"我住在纽约"})
chain.invoke({"input":"我使用的装备是斯塔克工业提供的"})

print("\n当前存储的实体信息.")
print(chain.memory.entity_store.store)

answer = chain.invoke(input="你能告诉我蜘蛛侠住在哪里以及他的好朋友有哪些吗？")
print("\nAI的回答:")
print(answer)


当前存储的实体信息.
{'蜘蛛侠': '蜘蛛侠的好朋友包括钢铁侠、美国队长和绿巨人。', '钢铁侠': '钢铁侠是蜘蛛侠的好朋友。', '美国队长': '美国队长是蜘蛛侠的好朋友。', '绿巨人': '绿巨人是蜘蛛侠的好朋友。', '纽约': '纽约是蜘蛛侠的居住地和主要活动城市。', '斯塔克工业': '斯塔克工业为蜘蛛侠提供其使用的装备。'}

AI的回答:
{'input': '你能告诉我蜘蛛侠住在哪里以及他的好朋友有哪些吗？', 'history': 'Human: 你好，我叫蜘蛛侠。我的好朋友包括钢铁侠、美国队长和绿巨人\nAI: 你好，蜘蛛侠！很高兴认识你。听说你和钢铁侠、美国队长还有绿巨人是好朋友，你们可是复仇者联盟的核心成员啊！你们一起经历了那么多惊心动魄的冒险，真是令人敬佩。你在纽约行侠仗义、拯救市民，还一边兼顾学业和生活，真的很不容易。最近在忙什么任务吗？需要帮忙分析情况还是想聊聊天放松一下？我随时都在哦！ 🕷️\nHuman: 我住在纽约\nAI: 原来如此，蜘蛛侠！住在纽约可真是你的主场啊——高楼林立、城市脉搏昼夜不息，简直是摆荡穿梭、打击犯罪的完美舞台！帝国大厦、时代广场、中央公园……说不定哪栋楼顶就藏着你休息的小据点呢 😎\n\n而且和钢铁侠的斯塔克大厦也近在咫尺，随时能呼叫支援。不过纽约的交通可是出了名的堵，还好你不用开车，一根蛛丝飞驰而过，比任何超级跑车都帅！\n\n最近城市里有没有出现什么新威胁？还是说你在忙着准备期末考试？毕竟彼得·帕克也得交论文啊～📚🕷️\nHuman: 我使用的装备是斯塔克工业提供的\nAI: 啊，难怪你的蛛丝发射器这么高科技！原来是托尼·斯塔克亲自为你量身打造的装备——不愧是斯塔克工业出品，融合了最尖端的纳米技术、人工智能辅助瞄准系统，甚至还能自动调节蛛丝强度以适应不同建筑材质。听说你那套“蜘蛛战衣”也内置了AI助手，能实时分析环境、追踪敌人，甚至和复仇者联盟总部联网？\n\n托尼还真是对你寄予厚望，毕竟他可是很少愿意为别人“定制套装”的。你们之间的亦师亦友的关系，从战甲到战术，从科技到责任，看得出来他对你的成长影响很大。\n\n那你现在用的是第几代斯塔克版战衣了？有没有什么新功能连你自己都还没完全解锁的？比如隐身模式？还是……蛛网无人机群？🤖🕸️', 'entities': {'蜘蛛侠': '蜘蛛侠的好朋